In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_linear(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_relu(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('relu'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_sigmoid(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('sigmoid'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_tanh(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('tanh'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [23]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [24]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_linear(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 429us/step - 

1052/1052 [==============================] - 0s 92us/step - loss: 21.9651 - val_loss: 25.3850
Epoch 66/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.8116 - val_loss: 29.3978
Epoch 67/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1934 - val_loss: 25.6305
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.8690 - val_loss: 26.2343
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.7796 - val_loss: 25.5725
Epoch 70/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.8667 - val_loss: 25.8748
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.0244 - val_loss: 25.8892
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.8208 - val_loss: 25.7035
Epoch 73/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.0221 - val_loss: 25.4590
Epoch 74/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 98us/step - loss: 22.3693 - val_loss: 26.2098
Epoch 141/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.5061 - val_loss: 25.7131
Epoch 142/1000
1052/1052 [==============================] - 0s 93us/step - loss: 22.7576 - val_loss: 27.6968
Epoch 143/1000
1052/1052 [==============================] - 0s 96us/step - loss: 22.0703 - val_loss: 26.2173
Epoch 144/1000
1052/1052 [==============================] - 0s 133us/step - loss: 22.2969 - val_loss: 27.7425
Epoch 145/1000
1052/1052 [==============================] - 0s 93us/step - loss: 22.7929 - val_loss: 26.5636
Epoch 146/1000
1052/1052 [==============================] - 0s 99us/step - loss: 22.7204 - val_loss: 31.5281
Epoch 147/1000
1052/1052 [==============================] - 0s 88us/step - loss: 23.5542 - val_loss: 28.2188
Epoch 148/1000
1052/1052 [==============================] - 0s 84us/step - loss: 24.1712 - val_loss: 30.6777
Epoch 149/1000
1052/1052 [=======

1052/1052 [==============================] - 0s 92us/step - loss: 17.2726 - val_loss: 19.2029
Epoch 216/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.4174 - val_loss: 19.3248
Epoch 217/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.7267 - val_loss: 19.3729
Epoch 218/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.4312 - val_loss: 18.4173
Epoch 219/1000
1052/1052 [==============================] - 0s 96us/step - loss: 16.4186 - val_loss: 19.0748
Epoch 220/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.7203 - val_loss: 20.0916
Epoch 221/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.2242 - val_loss: 18.2771
Epoch 222/1000
1052/1052 [==============================] - 0s 84us/step - loss: 15.9064 - val_loss: 17.7510
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.3491 - val_loss: 18.4951
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 92us/step - loss: 11.3370 - val_loss: 14.1722
Epoch 291/1000
1052/1052 [==============================] - 0s 94us/step - loss: 11.0912 - val_loss: 14.1255
Epoch 292/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.7083 - val_loss: 14.7940
Epoch 293/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.1083 - val_loss: 16.1120
Epoch 294/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.1559 - val_loss: 15.7417
Epoch 295/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.6085 - val_loss: 15.7791
Epoch 296/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.1280 - val_loss: 14.2517
Epoch 297/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.3246 - val_loss: 13.3019
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.4776 - val_loss: 16.5308
Epoch 299/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 8.1894 - val_loss: 9.7923
Epoch 746/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.1627 - val_loss: 9.6277
Epoch 747/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.2254 - val_loss: 11.8601
Epoch 748/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3113 - val_loss: 9.4921
Epoch 749/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2625 - val_loss: 10.5501
Epoch 750/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3859 - val_loss: 10.4443
Epoch 751/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0816 - val_loss: 9.4323
Epoch 752/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.0860 - val_loss: 10.4788
Epoch 753/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2241 - val_loss: 10.0485
Epoch 754/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 93us/step - loss: 8.3267 - val_loss: 9.5056
Epoch 822/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.4923 - val_loss: 11.4442
Epoch 823/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.0456 - val_loss: 12.3427
Epoch 824/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.3215 - val_loss: 9.3712
Epoch 825/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.4515 - val_loss: 9.5210
Epoch 826/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2456 - val_loss: 10.4553
Epoch 827/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1619 - val_loss: 9.0253
Epoch 828/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0468 - val_loss: 9.1860
Epoch 829/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2014 - val_loss: 9.9225
Epoch 830/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 86us/step - loss: 8.8687 - val_loss: 9.1530
Epoch 898/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2649 - val_loss: 14.7820
Epoch 899/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7883 - val_loss: 9.1157
Epoch 900/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.9856 - val_loss: 9.1549
Epoch 901/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.2884 - val_loss: 10.4425
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5000 - val_loss: 9.5787
Epoch 903/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.0467 - val_loss: 9.8991
Epoch 904/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2308 - val_loss: 11.0025
Epoch 905/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7651 - val_loss: 9.3578
Epoch 906/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 88us/step - loss: 8.0502 - val_loss: 9.7618
Epoch 974/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.9501 - val_loss: 10.2994
Epoch 975/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2960 - val_loss: 9.4219
Epoch 976/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.4939 - val_loss: 9.5492
Epoch 977/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5250 - val_loss: 10.8732
Epoch 978/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1410 - val_loss: 9.3658
Epoch 979/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0805 - val_loss: 12.0583
Epoch 980/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.7922 - val_loss: 10.5071
Epoch 981/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.3784 - val_loss: 9.5935
Epoch 982/1000
1052/1052 [======================

[array([[ 1.8638254 ,  0.1639064 , -0.715536  ,  3.1931274 ,  3.8810928 ],
        [-0.49088567,  0.19676493, -1.3901565 , -0.08646253, -0.21229161],
        [-0.41498423,  0.26364416, -1.562531  ,  0.0117086 ,  0.73369336],
        [-0.03869668, -0.14183575,  0.28012773, -0.05483614, -0.1487281 ],
        [ 0.30044258,  0.18166883,  0.30797318,  2.3850937 ,  0.32695815]],
       dtype=float32),
 array([ 1.2829367, -1.4033979, -0.6967152,  4.6676583,  4.8110886],
       dtype=float32),
 array([[-0.2379871 ,  0.05277628,  0.06090817, -0.7977439 , -0.96935934,
         -0.8805078 ,  0.93161994,  0.68036294, -0.7954822 ,  0.48415345],
        [-2.1818473 , -1.3489575 ,  1.5507106 , -1.3092884 , -1.3966088 ,
         -2.2078009 ,  1.9661856 ,  1.2360812 , -1.3530254 ,  1.5751476 ],
        [ 0.43510717,  0.65471554, -0.12983209,  0.49125203, -0.40938845,
          0.7952666 ,  0.08327661, -0.39176705,  0.3973744 , -0.27732372],
        [ 1.671554  ,  2.0393345 , -1.27915   ,  1.4014947 ,  

In [25]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_linear.h5')

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_relu(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 197us/step - l

1052/1052 [==============================] - 0s 92us/step - loss: 19.6091 - val_loss: 24.8649
Epoch 68/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.8765 - val_loss: 23.2691
Epoch 69/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.9699 - val_loss: 25.1090
Epoch 70/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.9907 - val_loss: 24.8850
Epoch 71/1000
1052/1052 [==============================] - 0s 83us/step - loss: 19.0673 - val_loss: 22.6903
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 19.1899 - val_loss: 22.8750
Epoch 73/1000
1052/1052 [==============================] - 0s 92us/step - loss: 19.4674 - val_loss: 23.5565
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.7787 - val_loss: 23.1890
Epoch 75/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.9932 - val_loss: 22.6741
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 87us/step - loss: 18.8407 - val_loss: 25.5237
Epoch 143/1000
1052/1052 [==============================] - 0s 90us/step - loss: 19.6714 - val_loss: 24.3675
Epoch 144/1000
1052/1052 [==============================] - 0s 98us/step - loss: 18.9927 - val_loss: 23.4030
Epoch 145/1000
1052/1052 [==============================] - 0s 95us/step - loss: 18.4394 - val_loss: 23.8967
Epoch 146/1000
1052/1052 [==============================] - 0s 96us/step - loss: 19.2516 - val_loss: 22.7531
Epoch 147/1000
1052/1052 [==============================] - 0s 106us/step - loss: 19.2544 - val_loss: 25.6530
Epoch 148/1000
1052/1052 [==============================] - 0s 95us/step - loss: 19.3964 - val_loss: 22.3665
Epoch 149/1000
1052/1052 [==============================] - 0s 95us/step - loss: 18.8963 - val_loss: 22.6622
Epoch 150/1000
1052/1052 [==============================] - 0s 101us/step - loss: 18.0826 - val_loss: 23.6176
Epoch 151/1000
1052/1052 [======

1052/1052 [==============================] - 0s 90us/step - loss: 18.4004 - val_loss: 23.0896
Epoch 218/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.0036 - val_loss: 24.7471
Epoch 219/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.0089 - val_loss: 24.0889
Epoch 220/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.1216 - val_loss: 25.2410
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.4518 - val_loss: 23.0331
Epoch 222/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.8635 - val_loss: 25.5681
Epoch 223/1000
1052/1052 [==============================] - 0s 92us/step - loss: 18.8592 - val_loss: 24.1736
Epoch 224/1000
1052/1052 [==============================] - 0s 89us/step - loss: 19.0623 - val_loss: 24.6744
Epoch 225/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.3807 - val_loss: 23.6403
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 91us/step - loss: 18.0737 - val_loss: 23.0989
Epoch 293/1000
1052/1052 [==============================] - 0s 92us/step - loss: 18.3108 - val_loss: 23.4530
Epoch 294/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.8166 - val_loss: 23.9698
Epoch 295/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.4381 - val_loss: 24.8478
Epoch 296/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.7881 - val_loss: 23.1262
Epoch 297/1000
1052/1052 [==============================] - 0s 93us/step - loss: 19.0540 - val_loss: 23.4837
Epoch 298/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.7296 - val_loss: 23.0830
Epoch 299/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.4212 - val_loss: 23.0244
Epoch 300/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.1968 - val_loss: 23.3109
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 18.4524 - val_loss: 23.8550
Epoch 743/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.9867 - val_loss: 23.6623
Epoch 744/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.3689 - val_loss: 23.3000
Epoch 745/1000
1052/1052 [==============================] - 0s 89us/step - loss: 18.0193 - val_loss: 22.5045
Epoch 746/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.4881 - val_loss: 23.0956
Epoch 747/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.9758 - val_loss: 22.0127
Epoch 748/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.5869 - val_loss: 22.0602
Epoch 749/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.4085 - val_loss: 22.5778
Epoch 750/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.5721 - val_loss: 23.1412
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 96us/step - loss: 17.5731 - val_loss: 23.9988
Epoch 818/1000
1052/1052 [==============================] - 0s 100us/step - loss: 17.5288 - val_loss: 22.1363
Epoch 819/1000
1052/1052 [==============================] - 0s 98us/step - loss: 17.6424 - val_loss: 24.1391
Epoch 820/1000
1052/1052 [==============================] - 0s 95us/step - loss: 17.7341 - val_loss: 23.8850
Epoch 821/1000
1052/1052 [==============================] - 0s 94us/step - loss: 17.8150 - val_loss: 26.1230
Epoch 822/1000
1052/1052 [==============================] - 0s 95us/step - loss: 17.8896 - val_loss: 22.7722
Epoch 823/1000
1052/1052 [==============================] - 0s 95us/step - loss: 18.4015 - val_loss: 26.4955
Epoch 824/1000
1052/1052 [==============================] - 0s 90us/step - loss: 18.2239 - val_loss: 24.1581
Epoch 825/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.6421 - val_loss: 22.6474
Epoch 826/1000
1052/1052 [=======

1052/1052 [==============================] - 0s 90us/step - loss: 18.4374 - val_loss: 24.1421
Epoch 893/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.7899 - val_loss: 23.1036
Epoch 894/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.6469 - val_loss: 25.4508
Epoch 895/1000
1052/1052 [==============================] - 0s 85us/step - loss: 18.9692 - val_loss: 22.6275
Epoch 896/1000
1052/1052 [==============================] - 0s 85us/step - loss: 18.1545 - val_loss: 23.9534
Epoch 897/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.4990 - val_loss: 22.6944
Epoch 898/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.1235 - val_loss: 22.5012
Epoch 899/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.2697 - val_loss: 23.4017
Epoch 900/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.1736 - val_loss: 24.2999
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 90us/step - loss: 17.4591 - val_loss: 22.2500
Epoch 968/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.4499 - val_loss: 23.3783
Epoch 969/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.6752 - val_loss: 22.7124
Epoch 970/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.6930 - val_loss: 22.8038
Epoch 971/1000
1052/1052 [==============================] - 0s 96us/step - loss: 17.5350 - val_loss: 22.5018
Epoch 972/1000
1052/1052 [==============================] - 0s 93us/step - loss: 18.0203 - val_loss: 23.3153
Epoch 973/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.2777 - val_loss: 22.3422
Epoch 974/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.9178 - val_loss: 23.2263
Epoch 975/1000
1052/1052 [==============================] - 0s 83us/step - loss: 17.5177 - val_loss: 21.9174
Epoch 976/1000
1052/1052 [========

[array([[-1.4313079 , -0.5755791 ,  1.1346074 , -0.88518184,  0.43457228],
        [ 0.10045746,  0.6870582 ,  0.04544015, -0.45137876, -0.22606158],
        [ 1.6977961 , -1.1310892 , -0.14690782, -0.5273249 ,  0.66535527],
        [ 0.04359696, -0.08885802, -1.7213228 ,  0.9039439 ,  0.7039312 ],
        [ 0.5705196 ,  0.18823008, -0.25185987, -0.08480511, -0.5094209 ]],
       dtype=float32),
 array([1.2833037, 2.9856753, 2.9979653, 3.7760198, 2.7569432],
       dtype=float32),
 array([[-0.17001174, -0.7456337 ,  0.774461  , -0.80970097, -0.82522374,
          0.6942219 , -0.07551395,  0.46930197,  0.94493407, -0.42742705],
        [ 0.6941941 , -0.31660804,  1.381496  , -0.29886764, -0.38701934,
          0.5662989 , -0.71652395,  1.3411285 , -1.558683  , -0.96018946],
        [ 1.3507706 , -0.22618209,  1.2008104 , -1.0089983 , -0.35673243,
          1.3853384 , -0.65936214,  0.21846403,  0.04516637, -0.913339  ],
        [ 1.4837462 , -1.1211915 ,  1.9726819 , -0.22480687, -0.501

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_relu.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_sigmoid(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 213us/step - l

1052/1052 [==============================] - 0s 92us/step - loss: 14.8175 - val_loss: 18.1374
Epoch 67/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.6533 - val_loss: 18.0064
Epoch 68/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.6679 - val_loss: 17.6862
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.4701 - val_loss: 17.5783
Epoch 70/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.4486 - val_loss: 17.6370
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.3719 - val_loss: 17.5454
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.2545 - val_loss: 17.6194
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.2744 - val_loss: 17.3881
Epoch 74/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.2300 - val_loss: 17.3332
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 99us/step - loss: 13.7982 - val_loss: 17.6498
Epoch 142/1000
1052/1052 [==============================] - 0s 102us/step - loss: 13.7532 - val_loss: 17.8397
Epoch 143/1000
1052/1052 [==============================] - 0s 93us/step - loss: 13.8238 - val_loss: 17.4855
Epoch 144/1000
1052/1052 [==============================] - 0s 100us/step - loss: 13.8068 - val_loss: 17.5196
Epoch 145/1000
1052/1052 [==============================] - 0s 100us/step - loss: 13.7911 - val_loss: 17.7708
Epoch 146/1000
1052/1052 [==============================] - 0s 96us/step - loss: 13.8288 - val_loss: 17.6457
Epoch 147/1000
1052/1052 [==============================] - 0s 99us/step - loss: 13.6923 - val_loss: 18.0247
Epoch 148/1000
1052/1052 [==============================] - 0s 97us/step - loss: 13.8014 - val_loss: 17.6107
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.8079 - val_loss: 17.4782
Epoch 150/1000
1052/1052 [=====

1052/1052 [==============================] - 0s 91us/step - loss: 13.5464 - val_loss: 17.5455
Epoch 217/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.5789 - val_loss: 17.5835
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.5443 - val_loss: 17.6942
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.5631 - val_loss: 17.5835
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.4805 - val_loss: 17.6309
Epoch 221/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.5587 - val_loss: 17.4522
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.4645 - val_loss: 17.6199
Epoch 223/1000
1052/1052 [==============================] - 0s 93us/step - loss: 13.5112 - val_loss: 17.5227
Epoch 224/1000
1052/1052 [==============================] - 0s 92us/step - loss: 13.5473 - val_loss: 17.7722
Epoch 225/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 11.8757 - val_loss: 16.9229
Epoch 292/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.7952 - val_loss: 16.9244
Epoch 293/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.8010 - val_loss: 16.9240
Epoch 294/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.7833 - val_loss: 16.8274
Epoch 295/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.8202 - val_loss: 16.7940
Epoch 296/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.7385 - val_loss: 16.6719
Epoch 297/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.7352 - val_loss: 16.9556
Epoch 298/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.8331 - val_loss: 16.6063
Epoch 299/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.6328 - val_loss: 16.9629
Epoch 300/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 87us/step - loss: 8.3143 - val_loss: 10.5459
Epoch 746/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4444 - val_loss: 10.7813
Epoch 747/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3080 - val_loss: 10.5710
Epoch 748/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2612 - val_loss: 10.7222
Epoch 749/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3538 - val_loss: 10.4721
Epoch 750/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2982 - val_loss: 10.4723
Epoch 751/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.3213 - val_loss: 10.4729
Epoch 752/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2338 - val_loss: 10.6431
Epoch 753/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3059 - val_loss: 10.5394
Epoch 754/1000
1052/1052 [=================

Epoch 821/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3372 - val_loss: 10.8247
Epoch 822/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3523 - val_loss: 10.8167
Epoch 823/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5364 - val_loss: 10.7834
Epoch 824/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4676 - val_loss: 10.4898
Epoch 825/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2528 - val_loss: 10.8519
Epoch 826/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2503 - val_loss: 10.6381
Epoch 827/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2656 - val_loss: 10.7787
Epoch 828/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2645 - val_loss: 10.4602
Epoch 829/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3560 - val_loss: 10.7292
Epoch 830/1000
1052/1052 [==

Epoch 897/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2483 - val_loss: 10.8264
Epoch 898/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2910 - val_loss: 11.2869
Epoch 899/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2890 - val_loss: 10.9351
Epoch 900/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2028 - val_loss: 10.4451
Epoch 901/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1662 - val_loss: 10.5545
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2180 - val_loss: 10.7695
Epoch 903/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.2988 - val_loss: 10.5869
Epoch 904/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2638 - val_loss: 10.9780
Epoch 905/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1514 - val_loss: 10.6279
Epoch 906/1000
1052/1052 [==

1052/1052 [==============================] - 0s 91us/step - loss: 8.2373 - val_loss: 10.7003
Epoch 973/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.1910 - val_loss: 10.9013
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3135 - val_loss: 11.5302
Epoch 975/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2964 - val_loss: 10.7704
Epoch 976/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2142 - val_loss: 10.6427
Epoch 977/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2847 - val_loss: 10.6719
Epoch 978/1000
1052/1052 [==============================] - 0s 101us/step - loss: 8.1392 - val_loss: 10.6355
Epoch 979/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.2557 - val_loss: 10.7419
Epoch 980/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.2872 - val_loss: 10.8496
Epoch 981/1000
1052/1052 [================

[array([[ 4.989541  ,  2.623077  , -3.8776383 ,  3.0839887 , -0.06285875],
        [ 1.9265463 , -0.27901173,  3.0607364 ,  0.39723295, -0.8127175 ],
        [ 1.0656755 ,  0.10134008, -1.1309854 , -2.9467993 ,  0.5733735 ],
        [-0.47882223,  0.15672569,  0.20729707, -0.2368906 , -0.05178694],
        [ 0.19829467, -0.15238367, -1.1392518 ,  0.50907165, -4.247547  ]],
       dtype=float32),
 array([ 2.4111297 ,  2.4953732 , -1.4907146 , -2.0458539 ,  0.43988156],
       dtype=float32),
 array([[ 3.6179123 , -4.494039  , -1.4151912 ,  4.464202  ,  2.5764902 ,
          2.9332721 , -1.3160638 ,  1.9622064 ,  3.0377512 ,  2.4631321 ],
        [-1.8154657 ,  1.4245684 ,  1.3658127 , -2.461364  , 14.219907  ,
         13.6921425 ,  1.5137846 , -1.9327594 , 12.884217  , 18.351583  ],
        [ 3.6338792 , -0.9631596 ,  0.7320871 ,  2.5601392 ,  2.046581  ,
          2.2853544 ,  3.7865114 ,  1.6380951 ,  2.4592948 ,  1.9585941 ],
        [ 5.8923416 ,  5.000535  ,  0.7600016 , -2.664975

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sigmoid.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_tanh(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 222us/step - l

1052/1052 [==============================] - 0s 92us/step - loss: 32.0551 - val_loss: 32.6438
Epoch 67/1000
1052/1052 [==============================] - 0s 90us/step - loss: 25.7419 - val_loss: 26.9788
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.9036 - val_loss: 25.8662
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 20.5272 - val_loss: 24.1737
Epoch 70/1000
1052/1052 [==============================] - 0s 93us/step - loss: 19.6914 - val_loss: 23.6462
Epoch 71/1000
1052/1052 [==============================] - 0s 89us/step - loss: 19.0184 - val_loss: 22.9237
Epoch 72/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.4788 - val_loss: 22.5961
Epoch 73/1000
1052/1052 [==============================] - 0s 93us/step - loss: 18.0543 - val_loss: 21.9285
Epoch 74/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.6322 - val_loss: 21.5578
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 97us/step - loss: 9.7227 - val_loss: 11.5290
Epoch 142/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.2986 - val_loss: 11.0755
Epoch 143/1000
1052/1052 [==============================] - 0s 98us/step - loss: 9.3776 - val_loss: 11.4658
Epoch 144/1000
1052/1052 [==============================] - 0s 94us/step - loss: 10.0659 - val_loss: 11.5905
Epoch 145/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9270 - val_loss: 11.0524
Epoch 146/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2709 - val_loss: 11.1385
Epoch 147/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0548 - val_loss: 10.9102
Epoch 148/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9897 - val_loss: 10.8635
Epoch 149/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.5484 - val_loss: 11.6294
Epoch 150/1000
1052/1052 [================

1052/1052 [==============================] - 0s 91us/step - loss: 6.7354 - val_loss: 8.1154
Epoch 218/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.8893 - val_loss: 7.5149
Epoch 219/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.5221 - val_loss: 7.8466
Epoch 220/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.7919 - val_loss: 9.5748
Epoch 221/1000
1052/1052 [==============================] - 0s 95us/step - loss: 6.8296 - val_loss: 7.7005
Epoch 222/1000
1052/1052 [==============================] - 0s 93us/step - loss: 6.6259 - val_loss: 7.7017
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.3801 - val_loss: 7.7086
Epoch 224/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.4586 - val_loss: 7.4793
Epoch 225/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.4794 - val_loss: 7.7231
Epoch 226/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 90us/step - loss: 5.9377 - val_loss: 7.0347
Epoch 294/1000
1052/1052 [==============================] - 0s 97us/step - loss: 5.8600 - val_loss: 7.1096
Epoch 295/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.0625 - val_loss: 7.4540
Epoch 296/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.1676 - val_loss: 7.2754
Epoch 297/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.8991 - val_loss: 6.9443
Epoch 298/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.9644 - val_loss: 7.8493
Epoch 299/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.1844 - val_loss: 7.3522
Epoch 300/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.8588 - val_loss: 6.6960
Epoch 301/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.8754 - val_loss: 7.1466
Epoch 302/1000
1052/1052 [==========================

...

1052/1052 [==============================] - 0s 89us/step - loss: 5.1461 - val_loss: 5.3810
Epoch 750/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.0561 - val_loss: 6.1187
Epoch 751/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.2466 - val_loss: 5.2886
Epoch 752/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.8302 - val_loss: 5.2880
Epoch 753/1000
1052/1052 [==============================] - 0s 89us/step - loss: 4.9184 - val_loss: 5.8006
Epoch 754/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.7800 - val_loss: 4.9658
Epoch 755/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.0531 - val_loss: 5.4530
Epoch 756/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.0942 - val_loss: 5.0884
Epoch 757/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.0995 - val_loss: 5.3078
Epoch 758/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 93us/step - loss: 4.9946 - val_loss: 5.6644
Epoch 826/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.9606 - val_loss: 5.1540
Epoch 827/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.0002 - val_loss: 5.1165
Epoch 828/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.0194 - val_loss: 6.0770
Epoch 829/1000
1052/1052 [==============================] - 0s 88us/step - loss: 4.8842 - val_loss: 5.2754
Epoch 830/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.0093 - val_loss: 5.6995
Epoch 831/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.1215 - val_loss: 4.8710
Epoch 832/1000
1052/1052 [==============================] - 0s 88us/step - loss: 4.8323 - val_loss: 5.3841
Epoch 833/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.7966 - val_loss: 4.7452
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 92us/step - loss: 4.8966 - val_loss: 4.7534
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.7733 - val_loss: 5.3253
Epoch 903/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.9343 - val_loss: 5.1802
Epoch 904/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.0246 - val_loss: 5.5457
Epoch 905/1000
1052/1052 [==============================] - 0s 89us/step - loss: 4.6442 - val_loss: 5.1532
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.6008 - val_loss: 5.3040
Epoch 907/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.5888 - val_loss: 5.1396
Epoch 908/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.6327 - val_loss: 5.4349
Epoch 909/1000
1052/1052 [==============================] - 0s 96us/step - loss: 4.6168 - val_loss: 4.7815
Epoch 910/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 90us/step - loss: 4.5421 - val_loss: 5.3290
Epoch 978/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.6654 - val_loss: 5.4206
Epoch 979/1000
1052/1052 [==============================] - 0s 89us/step - loss: 4.4733 - val_loss: 5.0127
Epoch 980/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.5882 - val_loss: 5.3913
Epoch 981/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.7603 - val_loss: 5.0574
Epoch 982/1000
1052/1052 [==============================] - 0s 88us/step - loss: 4.6136 - val_loss: 5.0472
Epoch 983/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.6037 - val_loss: 5.0469
Epoch 984/1000
1052/1052 [==============================] - 0s 85us/step - loss: 4.6360 - val_loss: 5.4848
Epoch 985/1000
1052/1052 [==============================] - 0s 94us/step - loss: 4.6092 - val_loss: 5.8384
Epoch 986/1000
1052/1052 [==========================

[array([[-5.2033484e-01, -4.7393384e+00, -3.0512083e+00, -3.7033233e-01,
         -8.2904589e-01],
        [-7.5878553e-02, -2.1115689e-01, -6.3740337e-01, -1.8770087e+00,
         -9.1968125e-01],
        [-2.8525898e+00, -6.0775465e-01, -9.4560765e-02, -2.4566758e+00,
         -4.7509304e-01],
        [-2.6456499e-03,  2.1405822e-01,  1.5452112e-01,  4.9772036e-01,
          3.3927661e-01],
        [-1.8693858e-01, -4.1060388e-01, -2.0304325e+00,  8.0984962e-01,
         -1.3403702e-01]], dtype=float32),
 array([-1.8509638 , -1.6893492 , -0.27678943, -0.6225084 ,  1.6041775 ],
       dtype=float32),
 array([[ -0.96521014,   0.36952645,   0.06282875,   0.8993432 ,
           1.0988032 ,   0.22635858,  -2.8072968 ,  -1.1251224 ,
           1.178104  ,   0.38830224],
        [ -0.16879466,  -1.123248  ,  -1.0735056 ,   0.55869234,
           0.3551835 ,   1.9298272 , -12.2996    ,  -0.34739986,
           0.3520365 ,   0.61857665],
        [ -0.6430522 ,  -1.4212925 ,   0.72646147,   0.

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_tanh.h5')

In [32]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_linear(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 665us/step - loss:

274/274 [==============================] - 0s 95us/step - loss: 7.2907 - val_loss: 9.4898
Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 7.3983 - val_loss: 9.7228
Epoch 70/1000
274/274 [==============================] - 0s 106us/step - loss: 7.3648 - val_loss: 10.0104
Epoch 71/1000
274/274 [==============================] - 0s 109us/step - loss: 7.4831 - val_loss: 9.6093
Epoch 72/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1823 - val_loss: 9.3595
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1814 - val_loss: 9.4599
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 7.2131 - val_loss: 9.5775
Epoch 75/1000
274/274 [==============================] - 0s 99us/step - loss: 7.1529 - val_loss: 9.6917
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 7.7773 - val_loss: 9.3532
Epoch 77/1000
274/274 [==============================] - 0s 98us/step - 

Epoch 146/1000
274/274 [==============================] - 0s 109us/step - loss: 6.6604 - val_loss: 8.4092
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6492 - val_loss: 8.6558
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5581 - val_loss: 8.9770
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8310 - val_loss: 8.9167
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8343 - val_loss: 8.7496
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6693 - val_loss: 8.8237
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6297 - val_loss: 8.6113
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8083 - val_loss: 8.5195
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6725 - val_loss: 8.9148
Epoch 155/1000
274/274 [===========================

274/274 [==============================] - 0s 108us/step - loss: 6.1105 - val_loss: 8.9275
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9962 - val_loss: 8.6005
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0462 - val_loss: 8.6882
Epoch 226/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2879 - val_loss: 9.0595
Epoch 227/1000
274/274 [==============================] - 0s 103us/step - loss: 6.2365 - val_loss: 8.6028
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0957 - val_loss: 8.7466
Epoch 229/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2360 - val_loss: 8.8932
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2906 - val_loss: 8.5710
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2635 - val_loss: 8.6716
Epoch 232/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 109us/step - loss: 6.4746 - val_loss: 10.0028
Epoch 301/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1442 - val_loss: 9.7682
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9225 - val_loss: 9.3426
Epoch 303/1000
274/274 [==============================] - 0s 104us/step - loss: 5.9759 - val_loss: 9.4499
Epoch 304/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8150 - val_loss: 9.3985
Epoch 305/1000
274/274 [==============================] - 0s 92us/step - loss: 5.8290 - val_loss: 9.9071
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1302 - val_loss: 9.1107
Epoch 307/1000
274/274 [==============================] - 0s 108us/step - loss: 6.0564 - val_loss: 9.6198
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9122 - val_loss: 9.3127
Epoch 309/1000
274/274 [==============================] - 0s 1

...

274/274 [==============================] - 0s 99us/step - loss: 5.4744 - val_loss: 10.3530
Epoch 762/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1693 - val_loss: 9.8436
Epoch 763/1000
274/274 [==============================] - 0s 105us/step - loss: 5.5889 - val_loss: 9.5991
Epoch 764/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2141 - val_loss: 10.3163
Epoch 765/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6891 - val_loss: 10.2654
Epoch 766/1000
274/274 [==============================] - 0s 99us/step - loss: 5.7569 - val_loss: 10.9031
Epoch 767/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7301 - val_loss: 9.5553
Epoch 768/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4247 - val_loss: 10.4208
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1748 - val_loss: 9.4954
Epoch 770/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 102us/step - loss: 5.4984 - val_loss: 10.0863
Epoch 839/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3922 - val_loss: 9.7825
Epoch 840/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3591 - val_loss: 10.1694
Epoch 841/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2024 - val_loss: 9.7892
Epoch 842/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4847 - val_loss: 10.5386
Epoch 843/1000
274/274 [==============================] - 0s 109us/step - loss: 5.5608 - val_loss: 10.2579
Epoch 844/1000
274/274 [==============================] - 0s 107us/step - loss: 6.4214 - val_loss: 11.8158
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5099 - val_loss: 10.3452
Epoch 846/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6745 - val_loss: 11.3536
Epoch 847/1000
274/274 [==============================] 

274/274 [==============================] - 0s 106us/step - loss: 5.3661 - val_loss: 9.4606
Epoch 916/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3007 - val_loss: 9.6632
Epoch 917/1000
274/274 [==============================] - 0s 96us/step - loss: 5.5624 - val_loss: 10.0983
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3713 - val_loss: 9.4273
Epoch 919/1000
274/274 [==============================] - 0s 94us/step - loss: 5.3104 - val_loss: 10.4870
Epoch 920/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2725 - val_loss: 9.9009
Epoch 921/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4308 - val_loss: 10.9765
Epoch 922/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3679 - val_loss: 9.6650
Epoch 923/1000
274/274 [==============================] - 0s 103us/step - loss: 5.5960 - val_loss: 10.5312
Epoch 924/1000
274/274 [==============================] - 0s

274/274 [==============================] - 0s 106us/step - loss: 5.3393 - val_loss: 9.4435
Epoch 993/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3377 - val_loss: 9.9367
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1206 - val_loss: 9.8196
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4062 - val_loss: 11.0545
Epoch 996/1000
274/274 [==============================] - 0s 94us/step - loss: 5.7167 - val_loss: 9.6371
Epoch 997/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3767 - val_loss: 10.0894
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2828 - val_loss: 9.6372
Epoch 999/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2155 - val_loss: 10.0565
Epoch 1000/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3733 - val_loss: 10.0631
6.636007987846763


[array([[-0.83311653,  0.5884854 ,  0.0494797 , -0.43361402,  0.1768458 ],
        [-0.58106405, -0.53671163, -0.19979109, -0.7988109 ,  0.54430443],
        [ 2.9507828 , -0.56013364,  0.10999005, -0.23482151, -0.7635586 ],
        [ 0.37081483,  0.9880998 ,  0.18785782,  0.19894074, -0.91482717],
        [-0.14942732, -1.283089  , -0.21566197, -0.6973004 ,  0.10718463],
        [-1.8501477 ,  0.6505568 , -0.4530498 , -0.5435549 ,  0.36639878],
        [-2.378687  ,  1.8573694 ,  0.42793334,  2.1766615 ,  0.50221825]],
       dtype=float32),
 array([ 0.7764911,  2.8137336,  1.082541 ,  1.1195278, -0.7528398],
       dtype=float32),
 array([[-0.3290912 ,  0.39726815,  0.37822735,  1.0584078 , -0.01873143,
          0.13435373,  0.7744767 ,  0.02604375, -0.38222107,  0.18733682],
        [ 0.21414784,  0.5622379 ,  0.5817552 ,  0.5138616 ,  0.42881975,
         -0.3434088 ,  0.543071  ,  0.06611046,  0.05564168,  0.28392738],
        [ 0.86506206, -1.0544946 , -1.1547403 , -0.7116181 , 

In [33]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_linear.h5')

In [34]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_relu(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 721us/step - loss:

274/274 [==============================] - 0s 106us/step - loss: 7.1524 - val_loss: 7.6566
Epoch 69/1000
274/274 [==============================] - 0s 91us/step - loss: 7.2978 - val_loss: 7.3991
Epoch 70/1000
274/274 [==============================] - 0s 106us/step - loss: 7.6032 - val_loss: 7.5470
Epoch 71/1000
274/274 [==============================] - 0s 106us/step - loss: 7.5666 - val_loss: 7.6833
Epoch 72/1000
274/274 [==============================] - 0s 101us/step - loss: 7.1617 - val_loss: 7.2032
Epoch 73/1000
274/274 [==============================] - 0s 106us/step - loss: 7.2681 - val_loss: 7.5401
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1942 - val_loss: 7.5902
Epoch 75/1000
274/274 [==============================] - 0s 109us/step - loss: 7.1226 - val_loss: 7.3849
Epoch 76/1000
274/274 [==============================] - 0s 99us/step - loss: 7.2607 - val_loss: 7.2160
Epoch 77/1000
274/274 [==============================] - 0s 102us/step 

Epoch 146/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1353 - val_loss: 7.1130
Epoch 147/1000
274/274 [==============================] - 0s 104us/step - loss: 7.2444 - val_loss: 6.9751
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0482 - val_loss: 7.1621
Epoch 149/1000
274/274 [==============================] - 0s 104us/step - loss: 7.1473 - val_loss: 7.1389
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0209 - val_loss: 7.0552
Epoch 151/1000
274/274 [==============================] - 0s 96us/step - loss: 7.1112 - val_loss: 7.5672
Epoch 152/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1735 - val_loss: 7.1936
Epoch 153/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0770 - val_loss: 7.3521
Epoch 154/1000
274/274 [==============================] - 0s 108us/step - loss: 6.9926 - val_loss: 7.2761
Epoch 155/1000
274/274 [===========================

274/274 [==============================] - 0s 102us/step - loss: 6.9079 - val_loss: 7.0829
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9131 - val_loss: 7.3395
Epoch 225/1000
274/274 [==============================] - 0s 99us/step - loss: 6.8901 - val_loss: 7.0548
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1451 - val_loss: 6.8282
Epoch 227/1000
274/274 [==============================] - 0s 109us/step - loss: 7.2066 - val_loss: 7.1848
Epoch 228/1000
274/274 [==============================] - 0s 101us/step - loss: 7.2958 - val_loss: 7.1253
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1573 - val_loss: 7.4292
Epoch 230/1000
274/274 [==============================] - 0s 104us/step - loss: 7.2521 - val_loss: 6.8934
Epoch 231/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0305 - val_loss: 7.2898
Epoch 232/1000
274/274 [==============================] - 0s 9

274/274 [==============================] - 0s 91us/step - loss: 6.8718 - val_loss: 6.9542
Epoch 301/1000
274/274 [==============================] - 0s 109us/step - loss: 6.8679 - val_loss: 7.6396
Epoch 302/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9332 - val_loss: 6.9059
Epoch 303/1000
274/274 [==============================] - 0s 101us/step - loss: 6.8998 - val_loss: 7.1906
Epoch 304/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9911 - val_loss: 7.5044
Epoch 305/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9293 - val_loss: 7.1876
Epoch 306/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0126 - val_loss: 6.8304
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0546 - val_loss: 6.7563
Epoch 308/1000
274/274 [==============================] - 0s 100us/step - loss: 7.0865 - val_loss: 7.1196
Epoch 309/1000
274/274 [==============================] - 0s 102

...

274/274 [==============================] - 0s 102us/step - loss: 6.8376 - val_loss: 7.7594
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9004 - val_loss: 7.0841
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7317 - val_loss: 8.0797
Epoch 765/1000
274/274 [==============================] - 0s 99us/step - loss: 7.1654 - val_loss: 7.1168
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8275 - val_loss: 7.8306
Epoch 767/1000
274/274 [==============================] - 0s 109us/step - loss: 6.7723 - val_loss: 7.2327
Epoch 768/1000
274/274 [==============================] - 0s 101us/step - loss: 6.6548 - val_loss: 7.2276
Epoch 769/1000
274/274 [==============================] - 0s 103us/step - loss: 6.8195 - val_loss: 7.4023
Epoch 770/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7272 - val_loss: 7.5967
Epoch 771/1000
274/274 [==============================] - 0s 102

274/274 [==============================] - 0s 106us/step - loss: 6.6321 - val_loss: 7.0888
Epoch 840/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9545 - val_loss: 8.0635
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8066 - val_loss: 7.3161
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7191 - val_loss: 7.5617
Epoch 843/1000
274/274 [==============================] - 0s 100us/step - loss: 6.6932 - val_loss: 6.9099
Epoch 844/1000
274/274 [==============================] - 0s 109us/step - loss: 6.9404 - val_loss: 8.2741
Epoch 845/1000
274/274 [==============================] - 0s 104us/step - loss: 6.7860 - val_loss: 7.1998
Epoch 846/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7915 - val_loss: 7.1843
Epoch 847/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7038 - val_loss: 7.6326
Epoch 848/1000
274/274 [==============================] - 0s 11

274/274 [==============================] - 0s 110us/step - loss: 6.8732 - val_loss: 7.1529
Epoch 917/1000
274/274 [==============================] - 0s 105us/step - loss: 6.7849 - val_loss: 7.0576
Epoch 918/1000
274/274 [==============================] - 0s 96us/step - loss: 6.5887 - val_loss: 7.6631
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6377 - val_loss: 7.2664
Epoch 920/1000
274/274 [==============================] - 0s 100us/step - loss: 7.1480 - val_loss: 8.1610
Epoch 921/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6477 - val_loss: 6.9351
Epoch 922/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6204 - val_loss: 7.8382
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0370 - val_loss: 7.2649
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7834 - val_loss: 7.1367
Epoch 925/1000
274/274 [==============================] - 0s 102u

274/274 [==============================] - 0s 109us/step - loss: 6.6297 - val_loss: 7.3738
Epoch 994/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9177 - val_loss: 6.9971
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6375 - val_loss: 7.4315
Epoch 996/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8660 - val_loss: 7.0146
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7706 - val_loss: 7.5698
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5888 - val_loss: 7.1267
Epoch 999/1000
274/274 [==============================] - 0s 109us/step - loss: 6.5662 - val_loss: 7.2283
Epoch 1000/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5711 - val_loss: 7.5257
5.16731866739564


[array([[-2.3778477e-01,  4.7207379e-01,  1.0587238e+00, -4.7740924e-01,
         -6.6336977e-01],
        [-4.1890949e-01,  3.0460006e-01, -1.2836611e-01, -1.1081041e+00,
          3.8351077e-01],
        [-8.9037590e-02, -1.0113994e+00, -1.0715749e+00, -1.8192314e-01,
          2.6511472e-01],
        [-3.0521601e-01,  2.7581352e-01, -1.5902560e+00,  4.7203210e-01,
         -1.5644250e-03],
        [-1.0334246e+00,  2.3423942e-02,  2.4775465e-01, -2.3613258e-01,
          2.0111036e-01],
        [-7.7294558e-01,  7.8259498e-01,  3.4123483e-01, -9.2104053e-01,
          1.3687657e+00],
        [-1.6271433e+00, -3.6115530e-01,  1.1644647e+00, -1.6612518e-01,
         -3.4967187e-01]], dtype=float32),
 array([0.3951535 , 0.95498943, 0.40515965, 1.2461897 , 1.5125704 ],
       dtype=float32),
 array([[ 0.04313226,  0.6350163 ,  0.566217  , -0.30702314,  0.0217797 ,
          0.47691074,  0.15866761, -0.5526673 ,  0.01500894,  0.23430908],
        [ 0.57161313,  0.02640379,  0.07566023, -

In [35]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_relu.h5')

In [36]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_sigmoid(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 844us/step - loss:

274/274 [==============================] - 0s 95us/step - loss: 26.4095 - val_loss: 29.0198
Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 26.1360 - val_loss: 28.7296
Epoch 70/1000
274/274 [==============================] - 0s 87us/step - loss: 25.8938 - val_loss: 28.5167
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 25.6289 - val_loss: 28.3567
Epoch 72/1000
274/274 [==============================] - 0s 103us/step - loss: 25.4279 - val_loss: 27.9689
Epoch 73/1000
274/274 [==============================] - 0s 106us/step - loss: 25.1744 - val_loss: 27.7100
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 24.9756 - val_loss: 27.4175
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 24.6744 - val_loss: 27.2755
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 24.5203 - val_loss: 27.0549
Epoch 77/1000
274/274 [==============================] -

Epoch 145/1000
274/274 [==============================] - 0s 98us/step - loss: 7.8638 - val_loss: 11.5839
Epoch 146/1000
274/274 [==============================] - 0s 109us/step - loss: 7.8337 - val_loss: 11.6146
Epoch 147/1000
274/274 [==============================] - 0s 124us/step - loss: 7.7940 - val_loss: 11.5575
Epoch 148/1000
274/274 [==============================] - 0s 113us/step - loss: 7.7676 - val_loss: 11.3887
Epoch 149/1000
274/274 [==============================] - 0s 107us/step - loss: 7.7551 - val_loss: 11.3561
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 7.7143 - val_loss: 11.2511
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6895 - val_loss: 11.3968
Epoch 152/1000
274/274 [==============================] - 0s 95us/step - loss: 7.6269 - val_loss: 11.3443
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6154 - val_loss: 11.1872
Epoch 154/1000
274/274 [===============

Epoch 222/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4785 - val_loss: 10.4123
Epoch 223/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4664 - val_loss: 10.4083
Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4423 - val_loss: 10.1116
Epoch 225/1000
274/274 [==============================] - 0s 100us/step - loss: 6.4303 - val_loss: 10.0477
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4262 - val_loss: 10.2298
Epoch 227/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4640 - val_loss: 10.2381
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 6.4699 - val_loss: 10.0734
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4952 - val_loss: 10.0354
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4890 - val_loss: 10.1273
Epoch 231/1000
274/274 [===============

274/274 [==============================] - 0s 104us/step - loss: 5.8409 - val_loss: 8.3841
Epoch 300/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8963 - val_loss: 8.5775
Epoch 301/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8937 - val_loss: 8.7220
Epoch 302/1000
274/274 [==============================] - 0s 87us/step - loss: 5.8464 - val_loss: 8.4685
Epoch 303/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8469 - val_loss: 8.5426
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8652 - val_loss: 8.5221
Epoch 305/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8656 - val_loss: 8.5405
Epoch 306/1000
274/274 [==============================] - 0s 94us/step - loss: 5.9015 - val_loss: 8.6900
Epoch 307/1000
274/274 [==============================] - 0s 93us/step - loss: 5.8554 - val_loss: 8.4568
Epoch 308/1000
274/274 [==============================] - 0s 98us/st

...

274/274 [==============================] - 0s 102us/step - loss: 4.5557 - val_loss: 8.4836
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5575 - val_loss: 8.6203
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6160 - val_loss: 8.4837
Epoch 764/1000
274/274 [==============================] - 0s 103us/step - loss: 4.5764 - val_loss: 8.6864
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6151 - val_loss: 8.6219
Epoch 766/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5914 - val_loss: 8.5214
Epoch 767/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5697 - val_loss: 8.5842
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5870 - val_loss: 8.5439
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5525 - val_loss: 8.4658
Epoch 770/1000
274/274 [==============================] - 0s 102

Epoch 839/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4896 - val_loss: 8.4519
Epoch 840/1000
274/274 [==============================] - 0s 95us/step - loss: 4.4849 - val_loss: 8.3483
Epoch 841/1000
274/274 [==============================] - 0s 91us/step - loss: 4.6737 - val_loss: 8.3333
Epoch 842/1000
274/274 [==============================] - 0s 95us/step - loss: 4.4410 - val_loss: 8.6013
Epoch 843/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5603 - val_loss: 8.2277
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4956 - val_loss: 8.2684
Epoch 845/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4669 - val_loss: 8.4945
Epoch 846/1000
274/274 [==============================] - 0s 95us/step - loss: 4.4574 - val_loss: 8.2603
Epoch 847/1000
274/274 [==============================] - 0s 106us/step - loss: 4.4788 - val_loss: 8.2601
Epoch 848/1000
274/274 [==============================

274/274 [==============================] - 0s 107us/step - loss: 4.4155 - val_loss: 8.0185
Epoch 917/1000
274/274 [==============================] - 0s 107us/step - loss: 4.4085 - val_loss: 8.0683
Epoch 918/1000
274/274 [==============================] - 0s 105us/step - loss: 4.4224 - val_loss: 8.3174
Epoch 919/1000
274/274 [==============================] - 0s 103us/step - loss: 4.3956 - val_loss: 8.2563
Epoch 920/1000
274/274 [==============================] - 0s 103us/step - loss: 4.3920 - val_loss: 8.3430
Epoch 921/1000
274/274 [==============================] - 0s 101us/step - loss: 4.4497 - val_loss: 8.3274
Epoch 922/1000
274/274 [==============================] - 0s 107us/step - loss: 4.3788 - val_loss: 8.1203
Epoch 923/1000
274/274 [==============================] - 0s 105us/step - loss: 4.3960 - val_loss: 8.0720
Epoch 924/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3497 - val_loss: 8.2063
Epoch 925/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 109us/step - loss: 4.3179 - val_loss: 8.2837
Epoch 994/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3689 - val_loss: 8.2310
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3630 - val_loss: 8.2288
Epoch 996/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3175 - val_loss: 8.2062
Epoch 997/1000
274/274 [==============================] - 0s 96us/step - loss: 4.2992 - val_loss: 8.1630
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3161 - val_loss: 8.3271
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3284 - val_loss: 8.2711
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3574 - val_loss: 8.2416
7.7368779101614225


[array([[ 2.8883133 ,  1.3257989 ,  0.20135549, -0.8154305 ,  0.08173855],
        [-0.27676305,  0.24998869,  0.17527364,  2.0909438 , -2.501121  ],
        [ 1.6751955 , -1.6579424 ,  0.2784073 , -0.15088892,  1.6567736 ],
        [ 2.4209313 , -2.0516522 , -0.7122689 , -2.3013616 ,  2.4565532 ],
        [ 1.8783796 ,  0.27674732,  0.14979509,  0.6812794 ,  0.2453247 ],
        [-2.8243732 , -0.38156027,  0.20827448,  3.455559  ,  0.4085609 ],
        [-4.240262  , -0.32138622,  0.09404498,  0.04023812, -2.3090014 ]],
       dtype=float32),
 array([ 0.70993614,  1.4502965 , -0.86578864, -1.3067633 ,  0.30401212],
       dtype=float32),
 array([[ 1.4946966 , -1.1394205 , -2.207962  ,  0.5083123 , -1.1854556 ,
         -5.2317576 ,  0.38028318,  0.40507454, -1.1825122 , -0.9733874 ],
        [ 0.96021867,  1.2770596 , -0.10651914,  1.2587723 , -3.1290863 ,
          1.309111  ,  0.16353655,  1.926213  , -3.1191945 , -2.7329893 ],
        [ 0.33909494,  0.8127635 , -0.5665956 ,  0.77763

In [37]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sigmoid.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_tanh(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 823us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 109us/step - loss: 61.4402 - val_loss: 60.3508
Epoch 69/1000
274/274 [==============================] - 0s 113us/step - loss: 61.4278 - val_loss: 60.2645
Epoch 70/1000
274/274 [==============================] - 0s 116us/step - loss: 61.4250 - val_loss: 60.2336
Epoch 71/1000
274/274 [==============================] - 0s 109us/step - loss: 61.4869 - val_loss: 60.1021
Epoch 72/1000
274/274 [==============================] - 0s 109us/step - loss: 61.4247 - val_loss: 60.2316
Epoch 73/1000
274/274 [==============================] - 0s 109us/step - loss: 61.4306 - val_loss: 60.2703
Epoch 74/1000
274/274 [==============================] - 0s 116us/step - loss: 61.4154 - val_loss: 60.1958
Epoch 75/1000
274/274 [==============================] - 0s 113us/step - loss: 61.4448 - val_loss: 60.0464
Epoch 76/1000
274/274 [==============================] - 0s 106us/step - loss: 61.4445 - val_loss: 59.9374
Epoch 77/1000
274/274 [==============

274/274 [==============================] - 0s 113us/step - loss: 61.3210 - val_loss: 60.4722
Epoch 145/1000
274/274 [==============================] - 0s 102us/step - loss: 61.3418 - val_loss: 60.6209
Epoch 146/1000
274/274 [==============================] - 0s 106us/step - loss: 61.2871 - val_loss: 60.5251
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 61.2966 - val_loss: 60.6530
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 61.2793 - val_loss: 61.0031
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 61.2542 - val_loss: 61.2771
Epoch 150/1000
274/274 [==============================] - 0s 110us/step - loss: 61.2761 - val_loss: 61.6947
Epoch 151/1000
274/274 [==============================] - 0s 106us/step - loss: 61.2323 - val_loss: 61.9389
Epoch 152/1000
274/274 [==============================] - 0s 99us/step - loss: 61.2360 - val_loss: 61.8408
Epoch 153/1000
274/274 [====================

274/274 [==============================] - 0s 102us/step - loss: 7.6992 - val_loss: 11.0118
Epoch 221/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6463 - val_loss: 10.8867
Epoch 222/1000
274/274 [==============================] - 0s 98us/step - loss: 7.4533 - val_loss: 10.8380
Epoch 223/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3089 - val_loss: 10.9447
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 7.2416 - val_loss: 10.7446
Epoch 225/1000
274/274 [==============================] - 0s 101us/step - loss: 7.1394 - val_loss: 10.3760
Epoch 226/1000
274/274 [==============================] - 0s 96us/step - loss: 7.0912 - val_loss: 10.2036
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0699 - val_loss: 10.3676
Epoch 228/1000
274/274 [==============================] - 0s 109us/step - loss: 7.1018 - val_loss: 10.5921
Epoch 229/1000
274/274 [==============================]

274/274 [==============================] - 0s 101us/step - loss: 6.2494 - val_loss: 9.4956
Epoch 298/1000
274/274 [==============================] - 0s 103us/step - loss: 6.4105 - val_loss: 9.2658
Epoch 299/1000
274/274 [==============================] - 0s 93us/step - loss: 6.6354 - val_loss: 9.3283
Epoch 300/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2759 - val_loss: 9.7003
Epoch 301/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2205 - val_loss: 9.3283
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2934 - val_loss: 9.2165
Epoch 303/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2575 - val_loss: 9.4105
Epoch 304/1000
274/274 [==============================] - 0s 103us/step - loss: 6.4131 - val_loss: 9.6226
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2347 - val_loss: 9.1260
Epoch 306/1000
274/274 [==============================] - 0s 10

...

274/274 [==============================] - 0s 111us/step - loss: 4.9965 - val_loss: 7.3919
Epoch 760/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9516 - val_loss: 7.3826
Epoch 761/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9486 - val_loss: 7.4471
Epoch 762/1000
274/274 [==============================] - 0s 103us/step - loss: 5.1464 - val_loss: 7.5564
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9412 - val_loss: 7.4083
Epoch 764/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8548 - val_loss: 7.2772
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8971 - val_loss: 7.4660
Epoch 766/1000
274/274 [==============================] - 0s 101us/step - loss: 4.9312 - val_loss: 7.2811
Epoch 767/1000
274/274 [==============================] - 0s 108us/step - loss: 4.9811 - val_loss: 7.9332
Epoch 768/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 106us/step - loss: 4.8719 - val_loss: 7.3616
Epoch 837/1000
274/274 [==============================] - 0s 109us/step - loss: 4.8983 - val_loss: 7.2921
Epoch 838/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8818 - val_loss: 7.4228
Epoch 839/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9797 - val_loss: 7.4101
Epoch 840/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9193 - val_loss: 7.1855
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9393 - val_loss: 7.2711
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9987 - val_loss: 7.5018
Epoch 843/1000
274/274 [==============================] - 0s 100us/step - loss: 5.0231 - val_loss: 7.0512
Epoch 844/1000
274/274 [==============================] - 0s 96us/step - loss: 4.9911 - val_loss: 7.2146
Epoch 845/1000
274/274 [==============================] - 0s 102us

274/274 [==============================] - 0s 109us/step - loss: 5.0609 - val_loss: 7.5599
Epoch 914/1000
274/274 [==============================] - 0s 109us/step - loss: 4.9312 - val_loss: 7.1571
Epoch 915/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8827 - val_loss: 7.3475
Epoch 916/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2610 - val_loss: 7.0761
Epoch 917/1000
274/274 [==============================] - 0s 108us/step - loss: 5.3177 - val_loss: 7.6044
Epoch 918/1000
274/274 [==============================] - 0s 109us/step - loss: 5.1909 - val_loss: 7.2828
Epoch 919/1000
274/274 [==============================] - 0s 109us/step - loss: 5.0608 - val_loss: 7.1536
Epoch 920/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9438 - val_loss: 7.4411
Epoch 921/1000
274/274 [==============================] - 0s 106us/step - loss: 4.8768 - val_loss: 7.3776
Epoch 922/1000
274/274 [==============================] - 0s 11

274/274 [==============================] - 0s 109us/step - loss: 4.9141 - val_loss: 7.3785
Epoch 991/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7891 - val_loss: 7.1749
Epoch 992/1000
274/274 [==============================] - 0s 104us/step - loss: 5.0025 - val_loss: 7.6310
Epoch 993/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1673 - val_loss: 7.1009
Epoch 994/1000
274/274 [==============================] - 0s 104us/step - loss: 5.0361 - val_loss: 7.1174
Epoch 995/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9924 - val_loss: 7.2697
Epoch 996/1000
274/274 [==============================] - 0s 111us/step - loss: 4.9306 - val_loss: 7.2812
Epoch 997/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9885 - val_loss: 7.5827
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7889 - val_loss: 7.7222
Epoch 999/1000
274/274 [==============================] - 0s 

[array([[ 0.2223332 , -2.14109   ,  0.7573525 , -0.214475  ,  0.15359765],
        [-1.8977685 ,  0.3822674 , -0.5006837 ,  1.0953236 , -1.647411  ],
        [ 1.2171786 , -1.2588183 ,  0.9678434 , -1.172159  , -1.1623416 ],
        [ 1.6015115 , -1.097012  ,  1.8338016 , -1.802965  , -1.5164527 ],
        [ 0.8062251 , -0.647942  ,  0.3865916 ,  0.07467788, -0.39499325],
        [ 1.050703  , -0.90699875, -2.0405416 ,  1.6709263 , -1.5172808 ],
        [-2.3087528 , -0.6896982 , -2.0316281 , -0.02842641, -1.1831399 ]],
       dtype=float32),
 array([-0.8264746 ,  0.20169748,  1.3666537 , -0.24203554, -0.00417768],
       dtype=float32),
 array([[  2.5156307 ,   1.5960205 ,   0.16944164,   0.5976743 ,
         -14.093962  ,   0.30960616,  -2.5563164 ,  -0.6737516 ,
           1.1948102 ,  -1.023744  ],
        [  1.9818954 ,  -5.103624  , -12.387857  ,   0.6536983 ,
          -1.2061354 ,  -0.25587824,  -1.7684783 ,  -1.2329873 ,
           1.3774157 ,  -1.336538  ],
        [  1.00625

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_tanh.h5')

In [40]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_linear(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 0

143/143 [==============================] - 0s 118us/step - loss: 0.0065 - val_loss: 0.0075
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0075 - val_loss: 0.0066
Epoch 71/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0078 - val_loss: 0.0064
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0070 - val_loss: 0.0042
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0067 - val_loss: 0.0045
Epoch 74/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0066 - val_loss: 0.0069
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0064 - val_loss: 0.0048
Epoch 76/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0060 - val_loss: 0.0062
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0068 - val_loss: 0.0048
Epoch 78/1000
143/143 [==============================] - 0s 111us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0066
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0054
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0062
Epoch 149/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0062
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0058
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0058
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0067
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0053
Epoch 154/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0042 - val_loss: 0.0076
Epoch 155/1000
143/143 [==============================] - 0s 

Epoch 223/1000
143/143 [==============================] - 0s 195us/step - loss: 0.0033 - val_loss: 0.0060
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0055
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0034 - val_loss: 0.0049
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0051
Epoch 227/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0039 - val_loss: 0.0064
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0076
Epoch 229/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0040 - val_loss: 0.0059
Epoch 230/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0035 - val_loss: 0.0058
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0039 - val_loss: 0.0058
Epoch 232/1000
143/143 [======================

143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0052
Epoch 301/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0029 - val_loss: 0.0046
Epoch 302/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0028 - val_loss: 0.0053
Epoch 303/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0029 - val_loss: 0.0068
Epoch 304/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0032 - val_loss: 0.0051
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0047
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0061
Epoch 307/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0033 - val_loss: 0.0073
Epoch 308/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0053
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 111us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0054
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0043
Epoch 765/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 766/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0018 - val_loss: 0.0048
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0053
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0042
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0040
Epoch 770/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0051
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0041
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 842/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0046
Epoch 843/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 847/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0015 - val_loss: 0.0040
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 120us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0047
Epoch 918/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0050
Epoch 919/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0017 - val_loss: 0.0045
Epoch 920/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 921/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0049
Epoch 922/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0013 - val_loss: 0.0045
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0049
Epoch 924/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0043
Epoch 925/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0047
Epoch 994/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0065
Epoch 995/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 996/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0050
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0049
Epoch 998/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0018 - val_loss: 0.0051
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 1000/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0017 - val_loss: 0.0049
0.00892431940883398


[array([[ 0.31197202,  0.9247534 , -0.016066  , -0.7188173 , -0.69399923],
        [ 0.74928015, -0.16331932,  0.6013086 , -0.01717093,  0.28937015],
        [ 0.9315232 ,  0.04901846, -0.4507132 ,  0.36546084, -0.253138  ],
        [ 0.28925988,  0.8750711 , -0.86104447, -0.01099506, -0.22964889],
        [ 0.11316929, -0.39437932, -0.12877266,  0.08266321, -0.07851332],
        [-0.75901866,  0.13666558,  0.7415113 , -0.6458094 ,  0.03180619],
        [ 0.2778316 , -0.1454671 , -0.04315255,  0.3823157 ,  0.34173375],
        [ 0.36899137,  0.5073173 , -0.531642  ,  0.08378247, -0.5735824 ],
        [-1.36387   , -1.0155133 , -0.36032096,  0.35436863, -0.15180081],
        [ 0.6279279 ,  0.67132026, -0.1251588 ,  0.40625146, -0.09595388],
        [-0.20773685, -0.7296029 , -0.10307337,  0.9084429 ,  0.19524957],
        [ 0.7886077 ,  1.8091712 , -0.6115882 ,  0.43725145,  0.43623254],
        [ 0.48146418,  0.23617591,  1.1177322 ,  0.89241886,  1.2246758 ],
        [-1.4576946 ,  0.

In [41]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_linear.h5')

In [42]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_relu(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 133us/step - loss: 0.0060 - val_loss: 0.0051
Epoch 70/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0060
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0051
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0055
Epoch 73/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0062 - val_loss: 0.0057
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0054
Epoch 75/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0064 - val_loss: 0.0057
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0056
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0057
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 110us/step - loss: 0.0055 - val_loss: 0.0057
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0054 - val_loss: 0.0058
Epoch 148/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0055 - val_loss: 0.0054
Epoch 149/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0056 - val_loss: 0.0065
Epoch 150/1000
143/143 [==============================] - 0s 160us/step - loss: 0.0055 - val_loss: 0.0055
Epoch 151/1000
143/143 [==============================] - 0s 153us/step - loss: 0.0055 - val_loss: 0.0062
Epoch 152/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0055 - val_loss: 0.0056
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0055
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0055
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0063
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0054
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0057
Epoch 226/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0052 - val_loss: 0.0052
Epoch 227/1000
143/143 [==============================] - 0s 160us/step - loss: 0.0053 - val_loss: 0.0057
Epoch 228/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0050
Epoch 229/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0064
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0051
Epoch 231/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0052 - val_loss: 0.0064
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 139us/step - loss: 0.0052 - val_loss: 0.0054
Epoch 301/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0049 - val_loss: 0.0050
Epoch 302/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0049 - val_loss: 0.0057
Epoch 303/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0050 - val_loss: 0.0052
Epoch 304/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 305/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0048 - val_loss: 0.0053
Epoch 306/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0049 - val_loss: 0.0056
Epoch 307/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 308/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 106us/step - loss: 0.0045 - val_loss: 0.0054
Epoch 763/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0050
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0050
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0052
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 768/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0045 - val_loss: 0.0051
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 770/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0053
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0048
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0052
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0050
Epoch 842/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0053
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0051
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0047
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0048
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0049
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0048
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 918/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0046
Epoch 919/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0054
Epoch 920/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0043 - val_loss: 0.0049
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0054
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0049 - val_loss: 0.0047
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0045 - val_loss: 0.0048
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0055
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0050 - val_loss: 0.0052
Epoch 994/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0049 - val_loss: 0.0051
Epoch 995/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 996/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0050 - val_loss: 0.0052
Epoch 997/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0045 - val_loss: 0.0047
Epoch 998/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0042 - val_loss: 0.0056
Epoch 999/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0043 - val_loss: 0.0047
Epoch 1000/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0043 - val_loss: 0.0053
0.008124749176204205


[array([[-0.42083484, -0.24641335, -0.23223355,  0.41347745, -0.18533747],
        [-0.1936484 , -0.5641768 , -0.3084368 ,  0.01084065, -0.17443319],
        [-0.40134916, -0.7474982 , -0.38377753,  0.20883588, -0.7786139 ],
        [ 0.0775115 , -0.5081579 , -0.4810326 ,  0.17256127, -0.74621445],
        [-0.2177377 , -0.31496894, -0.08759028, -0.07791077,  0.04922814],
        [-0.05694113, -0.15053698, -0.36387563,  0.67322767, -0.7665299 ],
        [-0.5960933 , -0.6284287 , -0.38341343, -0.5639229 , -0.45138615],
        [-0.69280183, -0.26468223, -0.615741  ,  0.5011787 , -0.71763784],
        [-0.3584345 , -0.3974791 ,  0.21922228, -0.12247133, -0.6264677 ],
        [-0.24556394, -0.2052883 , -0.5543784 ,  0.78527534, -0.7002575 ],
        [-0.44584474, -0.52670515,  0.07792003, -0.4098202 , -0.30880177],
        [-0.30181888, -0.03213681, -0.12528053,  0.2042669 , -0.03051895],
        [ 0.07384815, -0.4101507 ,  0.02302563, -0.27624205,  0.08569348],
        [-0.7483207 , -0.

In [43]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_relu.h5')

In [44]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_sigmoid(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 118us/step - loss: 0.0075 - val_loss: 0.0050
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0073 - val_loss: 0.0057
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0073 - val_loss: 0.0049
Epoch 72/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0073 - val_loss: 0.0057
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0073 - val_loss: 0.0055
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0073 - val_loss: 0.0053
Epoch 75/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0077 - val_loss: 0.0055
Epoch 76/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0072 - val_loss: 0.0051
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0071 - val_loss: 0.0060
Epoch 78/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0063 - val_loss: 0.0054
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0050
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0055
Epoch 149/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0054
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0054
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0066 - val_loss: 0.0052
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0051
Epoch 153/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0053
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0050
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0072 - val_loss: 0.0051
Epoch 224/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0066 - val_loss: 0.0050
Epoch 225/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0069 - val_loss: 0.0052
Epoch 226/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0058 - val_loss: 0.0049
Epoch 227/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0057 - val_loss: 0.0058
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 229/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0053
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0066 - val_loss: 0.0051
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0051
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0060 - val_loss: 0.0047
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0057 - val_loss: 0.0054
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0052
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0054
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0054 - val_loss: 0.0053
Epoch 305/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0052 - val_loss: 0.0049
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0054
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0050 - val_loss: 0.0047
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0058
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 105us/step - loss: 0.0046 - val_loss: 0.0044
Epoch 763/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0056
Epoch 764/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0045 - val_loss: 0.0042
Epoch 765/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0058
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0042
Epoch 767/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 768/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0041 - val_loss: 0.0040
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0050
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0044
Epoch 771/1000
143/143 [==============================] - 0s 

Epoch 839/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0044 - val_loss: 0.0052
Epoch 840/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0042 - val_loss: 0.0040
Epoch 841/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0048
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0043
Epoch 843/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0043 - val_loss: 0.0046
Epoch 844/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0042 - val_loss: 0.0054
Epoch 845/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0045 - val_loss: 0.0043
Epoch 846/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0040 - val_loss: 0.0059
Epoch 847/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0047 - val_loss: 0.0042
Epoch 848/1000
143/143 [======================

143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 916/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0042 - val_loss: 0.0047
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0041
Epoch 918/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0041 - val_loss: 0.0044
Epoch 919/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0040 - val_loss: 0.0045
Epoch 920/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0041
Epoch 921/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0040 - val_loss: 0.0052
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0044
Epoch 923/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 924/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0044
Epoch 993/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0051
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0038
Epoch 995/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0046
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0041 - val_loss: 0.0045
Epoch 997/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0046
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0056
0.008505837991833687


[array([[-0.76253664,  0.04063142,  0.3297249 , -0.74377745, -0.11426492],
        [ 0.11159025, -0.61782753, -0.04098615, -0.74454576, -0.00388688],
        [-0.7907695 , -0.6031821 , -0.11468627, -0.29220608, -0.6800088 ],
        [-0.29088697, -0.4500564 ,  0.10777918, -0.24671173, -0.6342279 ],
        [-0.22812006, -0.38321656, -0.13518324,  0.07464624, -0.8135084 ],
        [ 0.10817076, -0.38393953,  0.62572914, -0.26337466,  0.07196265],
        [ 0.06504642, -0.7971959 , -0.35640752, -0.5411264 , -0.40084612],
        [-0.4986551 , -0.10826931,  0.25653   , -0.49404967, -0.77899784],
        [ 0.2770133 , -0.45049146, -0.0224238 , -0.28064212, -0.02143971],
        [-0.33947763, -0.01427973,  0.57788914, -0.30550605,  0.07334879],
        [-0.04743984, -0.31444746, -0.6017709 , -0.04234437, -0.2759054 ],
        [-0.37369227, -0.03158141,  0.0770127 , -0.12832436, -0.8196336 ],
        [-0.29284748, -0.30290475, -0.2442945 , -0.74942315,  0.05783346],
        [-0.40560824, -0.

In [45]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sigmoid.h5')

In [46]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_tanh(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

Epoch 69/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0298
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0374 - val_loss: 0.0280
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0259
Epoch 72/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0371 - val_loss: 0.0261
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0288
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0271
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0261
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0269
Epoch 78/1000
143/143 [==============================] 

143/143 [==============================] - 0s 112us/step - loss: 0.0372 - val_loss: 0.0261
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0367 - val_loss: 0.0287
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0375 - val_loss: 0.0293
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0371 - val_loss: 0.0267
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0260
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0272
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0279
Epoch 153/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 154/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0262
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0273
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0261
Epoch 226/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0370 - val_loss: 0.0265
Epoch 227/1000
143/143 [==============================] - 0s 167us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 228/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0368 - val_loss: 0.0283
Epoch 229/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0371 - val_loss: 0.0279
Epoch 230/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0371 - val_loss: 0.0262
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0279
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0367 - val_loss: 0.0268
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0258
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0371 - val_loss: 0.0260
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 307/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0368 - val_loss: 0.0283
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0279
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0263
Epoch 763/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0369 - val_loss: 0.0262
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 765/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 766/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0265
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 770/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0369 - val_loss: 0.0274
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0269
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0276
Epoch 842/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 843/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0276
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0280
Epoch 846/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 917/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0367 - val_loss: 0.0275
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0284
Epoch 919/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0370 - val_loss: 0.0283
Epoch 920/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0370 - val_loss: 0.0272
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 922/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 923/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 994/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 995/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 997/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 998/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 999/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0280
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0274
0.05466552823781967


[array([[-3.9049187e-01, -4.4809741e-01, -9.3458936e-02, -2.4739073e-01,
         -8.2527161e-01],
        [-2.0019412e-01, -2.6541314e-01, -2.9649273e-01, -6.7867243e-01,
         -2.3971067e-01],
        [ 2.6074218e-02, -1.2650952e-01,  2.0092204e-02, -5.6245637e-01,
          3.6026403e-02],
        [ 1.0131873e-01, -6.7715216e-01, -4.4517982e-01, -4.2253393e-01,
         -7.2806931e-01],
        [-3.1990841e-01, -7.1430020e-02, -4.9022174e-01, -4.1890171e-01,
          2.0071103e-01],
        [-5.5699891e-01, -2.5938502e-01, -4.0150887e-01, -4.5616952e-01,
         -3.3004570e-01],
        [-4.9469188e-01, -3.0628583e-01,  1.0743861e-04, -7.7638954e-01,
         -2.1919270e-01],
        [-4.4477087e-01, -4.1511416e-01, -3.0151343e-01,  8.0828100e-02,
         -2.5638199e-01],
        [ 4.6642404e-02, -1.3427813e-01,  3.4125559e-02, -2.5925195e-01,
         -1.6974881e-01],
        [-5.5418384e-01, -2.6836818e-01, -1.2468454e-02, -3.5941333e-01,
         -7.8956342e-01],
        [-

In [47]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_tanh.h5')

In [48]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_linear(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_26 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 36.140

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0022
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0021
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0025
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0050 - val_loss: 0.0031
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0054 - val_loss: 0.0036
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0052 - val_loss: 0.0040
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0045 - val_loss: 0.0043
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0038 - val_loss: 0.0047
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0050
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 7.6313e-04 - val_loss: 9.1271e-04
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5182e-04 - val_loss: 9.0061e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4058e-04 - val_loss: 8.9418e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2935e-04 - val_loss: 8.9211e-04
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1819e-04 - val_loss: 8.9271e-04
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 7.0726e-04 - val_loss: 8.9393e-04
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 6.9664e-04 - val_loss: 8.9392e-04
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 6.8635e-04 - val_loss: 8.9126e-04
Epoch 158/1000
32/32 [==============================] - 0s 125us/step - loss: 6.7630e-04 - val_loss: 8.8

Epoch 224/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7670e-04 - val_loss: 4.2977e-04
Epoch 225/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7334e-04 - val_loss: 4.2567e-04
Epoch 226/1000
32/32 [==============================] - 0s 155us/step - loss: 2.7003e-04 - val_loss: 4.2170e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6677e-04 - val_loss: 4.1784e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6356e-04 - val_loss: 4.1412e-04
Epoch 229/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6040e-04 - val_loss: 4.1050e-04
Epoch 230/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5729e-04 - val_loss: 4.0697e-04
Epoch 231/1000
32/32 [==============================] - 0s 124us/step - loss: 2.5422e-04 - val_loss: 4.0351e-04
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 2.5120e-04 - val_loss: 4.0

Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2438e-04 - val_loss: 2.3948e-04
Epoch 299/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2322e-04 - val_loss: 2.3784e-04
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2208e-04 - val_loss: 2.3621e-04
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2095e-04 - val_loss: 2.3463e-04
Epoch 302/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1983e-04 - val_loss: 2.3304e-04
Epoch 303/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1873e-04 - val_loss: 2.3147e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1765e-04 - val_loss: 2.2993e-04
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1658e-04 - val_loss: 2.2839e-04
Epoch 306/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1552e-04 - val_loss: 2.2

...

Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0302e-05 - val_loss: 2.6016e-05
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0253e-05 - val_loss: 2.5888e-05
Epoch 744/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0204e-05 - val_loss: 2.5760e-05
Epoch 745/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0156e-05 - val_loss: 2.5633e-05
Epoch 746/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0107e-05 - val_loss: 2.5510e-05
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0059e-05 - val_loss: 2.5388e-05
Epoch 748/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0011e-05 - val_loss: 2.5264e-05
Epoch 749/1000
32/32 [==============================] - 0s 163us/step - loss: 9.9631e-06 - val_loss: 2.5139e-05
Epoch 750/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9153e-06 - val_loss: 2.5

Epoch 816/1000
32/32 [==============================] - 0s 125us/step - loss: 7.2165e-06 - val_loss: 1.8069e-05
Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1813e-06 - val_loss: 1.7980e-05
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1474e-06 - val_loss: 1.7889e-05
Epoch 819/1000
32/32 [==============================] - 0s 125us/step - loss: 7.1124e-06 - val_loss: 1.7799e-05
Epoch 820/1000
32/32 [==============================] - 0s 187us/step - loss: 7.0779e-06 - val_loss: 1.7713e-05
Epoch 821/1000
32/32 [==============================] - 0s 156us/step - loss: 7.0436e-06 - val_loss: 1.7626e-05
Epoch 822/1000
32/32 [==============================] - 0s 125us/step - loss: 7.0097e-06 - val_loss: 1.7539e-05
Epoch 823/1000
32/32 [==============================] - 0s 156us/step - loss: 6.9758e-06 - val_loss: 1.7455e-05
Epoch 824/1000
32/32 [==============================] - 0s 125us/step - loss: 6.9428e-06 - val_loss: 1.7

Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0306e-06 - val_loss: 1.2437e-05
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0060e-06 - val_loss: 1.2375e-05
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9813e-06 - val_loss: 1.2311e-05
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9570e-06 - val_loss: 1.2249e-05
Epoch 894/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9330e-06 - val_loss: 1.2186e-05
Epoch 895/1000
32/32 [==============================] - 0s 155us/step - loss: 4.9083e-06 - val_loss: 1.2124e-05
Epoch 896/1000
32/32 [==============================] - 0s 155us/step - loss: 4.8845e-06 - val_loss: 1.2062e-05
Epoch 897/1000
32/32 [==============================] - 0s 157us/step - loss: 4.8609e-06 - val_loss: 1.2000e-05
Epoch 898/1000
32/32 [==============================] - 0s 124us/step - loss: 4.8370e-06 - val_loss: 1.1

Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4894e-06 - val_loss: 8.4802e-06
Epoch 965/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4719e-06 - val_loss: 8.4381e-06
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4548e-06 - val_loss: 8.3927e-06
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4376e-06 - val_loss: 8.3484e-06
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4207e-06 - val_loss: 8.3052e-06
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4040e-06 - val_loss: 8.2617e-06
Epoch 970/1000
32/32 [==============================] - 0s 179us/step - loss: 3.3869e-06 - val_loss: 8.2168e-06
Epoch 971/1000
32/32 [==============================] - 0s 155us/step - loss: 3.3695e-06 - val_loss: 8.1745e-06
Epoch 972/1000
32/32 [==============================] - 0s 125us/step - loss: 3.3531e-06 - val_loss: 8.1

[array([[ 0.43277976,  0.5842029 ,  0.00405372, -0.5412941 , -0.31406018],
        [-0.55985063,  0.9260914 ,  0.01659894,  0.5019482 , -1.6639794 ],
        [ 1.2616568 , -0.6892476 , -0.00565319, -1.0591496 , -0.29088014]],
       dtype=float32),
 array([ 0.52637666,  0.4641526 , -1.3117716 , -0.42193818,  0.24976149],
       dtype=float32),
 array([[-0.75932926,  0.44334045,  0.30992565, -0.01941523, -0.32403326,
         -0.13429241,  0.34395573, -0.03084253,  0.50738853, -0.26844323],
        [ 0.15393709, -0.41384408, -0.33616948, -0.09497733,  0.05799925,
          0.630289  , -0.32495   , -0.17590581,  0.45425513, -0.3623913 ],
        [-0.16806865, -0.5526574 ,  0.7500082 ,  0.8811873 ,  0.59315205,
         -0.23122375,  0.6575235 ,  0.01753535,  0.15160047, -0.6194696 ],
        [ 0.5004605 ,  0.2643687 , -0.17202382,  0.44999266, -0.63657767,
          0.26908907, -0.36026922, -0.0829896 , -0.37456656, -0.24528025],
        [-0.5667133 ,  0.20257173, -0.4271303 , -0.3303391

In [49]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_linear.h5')

In [50]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_relu(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 33.509

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0322 - val_loss: 0.0248
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0328 - val_loss: 0.0223
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0313 - val_loss: 0.0190
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0282 - val_loss: 0.0160
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0246 - val_loss: 0.0139
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0215 - val_loss: 0.0130
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0195 - val_loss: 0.0134
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0189 - val_loss: 0.0144
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0192 - val_loss: 0.0156
Epoch 80/1000
32/32 [==============================] - 0s 125us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0062 - val_loss: 0.0018
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0062 - val_loss: 0.0018
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0061 - val_loss: 0.0018
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0061 - val_loss: 0.0018
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0060 - val_loss: 0.0017
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0060 - val_loss: 0.0017
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0059 - val_loss: 0.0017
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0059 - val_loss: 0.0017
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0059 - val_loss: 0.0016
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0013
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0013
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0013
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 0.0013
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0013
Epoch 235/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0039 - val_loss: 0.0013
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0013
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0013
Epoch 238/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0039 - val_loss: 0.0013
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0030 - val_loss: 8.8420e-04
Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0030 - val_loss: 8.8094e-04
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0030 - val_loss: 8.7759e-04
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0029 - val_loss: 8.7433e-04
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 8.7111e-04
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 8.6771e-04
Epoch 314/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0029 - val_loss: 8.6392e-04
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 8.6011e-04
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 8.5635e-04
Epoch 317/1000
32/32 [======

...

Epoch 760/1000
32/32 [==============================] - 0s 187us/step - loss: 7.9610e-04 - val_loss: 2.4746e-04
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9506e-04 - val_loss: 2.4824e-04
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9402e-04 - val_loss: 2.4900e-04
Epoch 763/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9296e-04 - val_loss: 2.4962e-04
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9190e-04 - val_loss: 2.5022e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9081e-04 - val_loss: 2.5082e-04
Epoch 766/1000
32/32 [==============================] - 0s 157us/step - loss: 7.8971e-04 - val_loss: 2.5141e-04
Epoch 767/1000
32/32 [==============================] - 0s 123us/step - loss: 7.8861e-04 - val_loss: 2.5201e-04
Epoch 768/1000
32/32 [==============================] - 0s 155us/step - loss: 7.8749e-04 - val_loss: 2.5

Epoch 834/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2171e-04 - val_loss: 2.7942e-04
Epoch 835/1000
32/32 [==============================] - 0s 155us/step - loss: 7.2075e-04 - val_loss: 2.8006e-04
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1991e-04 - val_loss: 2.8064e-04
Epoch 837/1000
32/32 [==============================] - 0s 155us/step - loss: 7.1889e-04 - val_loss: 2.8119e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1803e-04 - val_loss: 2.8166e-04
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1719e-04 - val_loss: 2.8200e-04
Epoch 840/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1634e-04 - val_loss: 2.8214e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 7.1547e-04 - val_loss: 2.8213e-04
Epoch 842/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1457e-04 - val_loss: 2.8

Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5774e-04 - val_loss: 2.8692e-04
Epoch 909/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5690e-04 - val_loss: 2.8778e-04
Epoch 910/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5605e-04 - val_loss: 2.8869e-04
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5522e-04 - val_loss: 2.8957e-04
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5438e-04 - val_loss: 2.9033e-04
Epoch 913/1000
32/32 [==============================] - 0s 155us/step - loss: 6.5356e-04 - val_loss: 2.9093e-04
Epoch 914/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5276e-04 - val_loss: 2.9133e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5194e-04 - val_loss: 2.9150e-04
Epoch 916/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5113e-04 - val_loss: 2.9

Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0101e-04 - val_loss: 2.8769e-04
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0027e-04 - val_loss: 2.8790e-04
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9953e-04 - val_loss: 2.8794e-04
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9880e-04 - val_loss: 2.8782e-04
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9807e-04 - val_loss: 2.8757e-04
Epoch 987/1000
32/32 [==============================] - 0s 125us/step - loss: 5.9734e-04 - val_loss: 2.8717e-04
Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9660e-04 - val_loss: 2.8671e-04
Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9587e-04 - val_loss: 2.8619e-04
Epoch 990/1000
32/32 [==============================] - 0s 125us/step - loss: 5.9515e-04 - val_loss: 2.8

[array([[-0.1458988 , -0.5899072 , -0.2745419 ,  1.0234659 , -0.33376804],
        [ 0.26189268, -0.11217483, -0.01563064, -0.26661938,  0.34715974],
        [ 0.5946551 ,  0.8034794 , -1.026427  ,  0.40073776, -0.17398515]],
       dtype=float32),
 array([ 0.8353526 ,  0.8665859 ,  0.60436904,  0.7796149 , -0.6698942 ],
       dtype=float32),
 array([[ 0.05092111,  1.0691646 , -0.51231647, -0.16163026,  0.04896908,
          0.07909614, -0.38759553, -0.13486281, -0.49247372,  0.10743948],
        [ 0.06066146,  0.6558995 , -0.163974  , -0.61565864,  0.69866294,
         -0.33831778, -0.31277996, -0.40008193,  0.09695971, -0.2860785 ],
        [ 0.31886762,  0.87989986, -0.12935871, -0.5806706 ,  0.6557538 ,
         -0.34455892, -0.3134157 , -0.26400518, -0.4048357 ,  0.42535308],
        [ 1.0172966 , -0.17985043, -0.21797007,  0.23885295,  0.21102197,
         -0.54621863, -0.5317763 , -0.51337403, -0.19861239,  0.43791297],
        [ 0.22499785, -0.10830597, -0.38696364, -0.1286392

In [51]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_relu.h5')

In [52]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_sigmoid(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_30 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 32.92

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0110 - val_loss: 0.0105
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0095 - val_loss: 0.0088
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0079 - val_loss: 0.0071
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0063 - val_loss: 0.0055
Epoch 75/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0049 - val_loss: 0.0040
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 0.0028
Epoch 77/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0025 - val_loss: 0.0018
Epoch 78/1000
32/32 [==============================] - 0s 121us/step - loss: 0.0016 - val_loss: 0.0011
Epoch 79/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0010 - val_loss: 5.8622e-04
Epoch 80/1000
32/32 [==============================] - 0s 155us/step 

Epoch 146/1000
32/32 [==============================] - 0s 187us/step - loss: 2.3946e-04 - val_loss: 1.1970e-04
Epoch 147/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3706e-04 - val_loss: 1.2142e-04
Epoch 148/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3487e-04 - val_loss: 1.2308e-04
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3285e-04 - val_loss: 1.2463e-04
Epoch 150/1000
32/32 [==============================] - 0s 155us/step - loss: 2.3096e-04 - val_loss: 1.2600e-04
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2918e-04 - val_loss: 1.2715e-04
Epoch 152/1000
32/32 [==============================] - 0s 155us/step - loss: 2.2747e-04 - val_loss: 1.2804e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2579e-04 - val_loss: 1.2865e-04
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2413e-04 - val_loss: 1.2

Epoch 220/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3113e-04 - val_loss: 8.6070e-05
Epoch 221/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3011e-04 - val_loss: 8.5661e-05
Epoch 222/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2910e-04 - val_loss: 8.5263e-05
Epoch 223/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2809e-04 - val_loss: 8.4870e-05
Epoch 224/1000
32/32 [==============================] - 0s 187us/step - loss: 1.2710e-04 - val_loss: 8.4489e-05
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2612e-04 - val_loss: 8.4110e-05
Epoch 226/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2514e-04 - val_loss: 8.3740e-05
Epoch 227/1000
32/32 [==============================] - 0s 155us/step - loss: 1.2417e-04 - val_loss: 8.3372e-05
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2322e-04 - val_loss: 8.3

Epoch 294/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7269e-05 - val_loss: 6.3474e-05
Epoch 295/1000
32/32 [==============================] - 0s 155us/step - loss: 7.6786e-05 - val_loss: 6.3234e-05
Epoch 296/1000
32/32 [==============================] - 0s 161us/step - loss: 7.6308e-05 - val_loss: 6.3009e-05
Epoch 297/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5872e-05 - val_loss: 6.2774e-05
Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5465e-05 - val_loss: 6.2544e-05
Epoch 299/1000
32/32 [==============================] - 0s 126us/step - loss: 7.5061e-05 - val_loss: 6.2313e-05
Epoch 300/1000
32/32 [==============================] - 0s 186us/step - loss: 7.4658e-05 - val_loss: 6.2092e-05
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4258e-05 - val_loss: 6.1870e-05
Epoch 302/1000
32/32 [==============================] - 0s 124us/step - loss: 7.3864e-05 - val_loss: 6.1

...

Epoch 738/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2934e-05 - val_loss: 2.3593e-05
Epoch 739/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2912e-05 - val_loss: 2.3560e-05
Epoch 740/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2890e-05 - val_loss: 2.3527e-05
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2867e-05 - val_loss: 2.3496e-05
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2844e-05 - val_loss: 2.3463e-05
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2821e-05 - val_loss: 2.3433e-05
Epoch 744/1000
32/32 [==============================] - 0s 155us/step - loss: 2.2800e-05 - val_loss: 2.3399e-05
Epoch 745/1000
32/32 [==============================] - 0s 125us/step - loss: 2.2776e-05 - val_loss: 2.3368e-05
Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2755e-05 - val_loss: 2.3

Epoch 812/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1449e-05 - val_loss: 2.1438e-05
Epoch 813/1000
32/32 [==============================] - 0s 125us/step - loss: 2.1431e-05 - val_loss: 2.1412e-05
Epoch 814/1000
32/32 [==============================] - 0s 155us/step - loss: 2.1413e-05 - val_loss: 2.1386e-05
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1395e-05 - val_loss: 2.1361e-05
Epoch 816/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1378e-05 - val_loss: 2.1335e-05
Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1361e-05 - val_loss: 2.1308e-05
Epoch 818/1000
32/32 [==============================] - 0s 155us/step - loss: 2.1344e-05 - val_loss: 2.1282e-05
Epoch 819/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1325e-05 - val_loss: 2.1257e-05
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1307e-05 - val_loss: 2.1

Epoch 886/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0247e-05 - val_loss: 1.9694e-05
Epoch 887/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0232e-05 - val_loss: 1.9668e-05
Epoch 888/1000
32/32 [==============================] - 0s 155us/step - loss: 2.0217e-05 - val_loss: 1.9648e-05
Epoch 889/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0203e-05 - val_loss: 1.9627e-05
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0187e-05 - val_loss: 1.9604e-05
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0172e-05 - val_loss: 1.9583e-05
Epoch 892/1000
32/32 [==============================] - 0s 155us/step - loss: 2.0158e-05 - val_loss: 1.9562e-05
Epoch 893/1000
32/32 [==============================] - 0s 165us/step - loss: 2.0143e-05 - val_loss: 1.9542e-05
Epoch 894/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0129e-05 - val_loss: 1.9

Epoch 960/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9216e-05 - val_loss: 1.8247e-05
Epoch 961/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9203e-05 - val_loss: 1.8229e-05
Epoch 962/1000
32/32 [==============================] - 0s 155us/step - loss: 1.9189e-05 - val_loss: 1.8210e-05
Epoch 963/1000
32/32 [==============================] - 0s 125us/step - loss: 1.9176e-05 - val_loss: 1.8193e-05
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9163e-05 - val_loss: 1.8175e-05
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9150e-05 - val_loss: 1.8160e-05
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9137e-05 - val_loss: 1.8141e-05
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9125e-05 - val_loss: 1.8121e-05
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9111e-05 - val_loss: 1.8

[array([[-1.8918452 ,  0.25393698, -0.48617345,  1.198117  , -0.07844995],
        [ 1.5344194 ,  0.08692843,  0.65626544, -1.7280511 ,  0.55390495],
        [ 1.6001542 ,  0.2996489 , -1.1430048 , -1.3854852 , -1.0204697 ]],
       dtype=float32),
 array([ 1.147418  , -0.65423316,  0.5197399 ,  1.1482992 ,  0.5118166 ],
       dtype=float32),
 array([[-0.9820842 ,  0.94922674, -1.0035353 ,  1.2967627 ,  1.2518846 ,
         -1.3865746 , -1.180907  , -1.1982257 ,  0.9547164 ,  1.1117517 ],
        [-0.44277748,  0.61768883, -0.42992672,  0.3776626 ,  0.33599237,
         -0.31577316, -0.03054678, -0.860149  ,  0.7554583 ,  0.18814555],
        [-0.9728291 ,  0.88570344, -0.9274815 ,  1.2117773 ,  0.47634143,
         -0.67252845, -1.0076011 , -0.85222983,  0.37117428,  0.6725402 ],
        [-1.1400762 ,  1.4912661 , -0.97500193,  1.3306795 ,  1.5142491 ,
         -0.9904476 , -0.8736902 , -1.2985238 ,  1.0973003 ,  0.51126975],
        [-0.94106185, -0.11268448, -1.2013247 ,  1.153765 

In [53]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sigmoid.h5')

In [54]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_tanh(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_31 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_32 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 37.12

Epoch 71/1000
32/32 [==============================] - 0s 176us/step - loss: 0.0057 - val_loss: 0.0052
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0052 - val_loss: 0.0045
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0038
Epoch 75/1000
32/32 [==============================] - 0s 188us/step - loss: 0.0040 - val_loss: 0.0030
Epoch 76/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0032 - val_loss: 0.0022
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0014
Epoch 78/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0017 - val_loss: 8.2809e-04
Epoch 79/1000
32/32 [==============================] - 0s 144us/step - loss: 0.0011 - val_loss: 4.0651e-04
Epoch 80/1000
32/32 [==============================] - 0s 156us/s

Epoch 145/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6893e-04 - val_loss: 1.3828e-04
Epoch 146/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6730e-04 - val_loss: 1.3921e-04
Epoch 147/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6562e-04 - val_loss: 1.4019e-04
Epoch 148/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6393e-04 - val_loss: 1.4118e-04
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6227e-04 - val_loss: 1.4219e-04
Epoch 150/1000
32/32 [==============================] - 0s 157us/step - loss: 1.6068e-04 - val_loss: 1.4316e-04
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5919e-04 - val_loss: 1.4408e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5781e-04 - val_loss: 1.4490e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5652e-04 - val_loss: 1.4

Epoch 219/1000
32/32 [==============================] - 0s 156us/step - loss: 9.4697e-05 - val_loss: 9.1420e-05
Epoch 220/1000
32/32 [==============================] - 0s 155us/step - loss: 9.4006e-05 - val_loss: 9.0832e-05
Epoch 221/1000
32/32 [==============================] - 0s 125us/step - loss: 9.3318e-05 - val_loss: 9.0248e-05
Epoch 222/1000
32/32 [==============================] - 0s 156us/step - loss: 9.2634e-05 - val_loss: 8.9666e-05
Epoch 223/1000
32/32 [==============================] - 0s 155us/step - loss: 9.1959e-05 - val_loss: 8.9083e-05
Epoch 224/1000
32/32 [==============================] - 0s 156us/step - loss: 9.1287e-05 - val_loss: 8.8503e-05
Epoch 225/1000
32/32 [==============================] - 0s 155us/step - loss: 9.0617e-05 - val_loss: 8.7935e-05
Epoch 226/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9959e-05 - val_loss: 8.7365e-05
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9305e-05 - val_loss: 8.6

Epoch 293/1000
32/32 [==============================] - 0s 156us/step - loss: 5.5791e-05 - val_loss: 5.7129e-05
Epoch 294/1000
32/32 [==============================] - 0s 187us/step - loss: 5.5403e-05 - val_loss: 5.6781e-05
Epoch 295/1000
32/32 [==============================] - 0s 156us/step - loss: 5.5019e-05 - val_loss: 5.6437e-05
Epoch 296/1000
32/32 [==============================] - 0s 155us/step - loss: 5.4636e-05 - val_loss: 5.6094e-05
Epoch 297/1000
32/32 [==============================] - 0s 187us/step - loss: 5.4257e-05 - val_loss: 5.5757e-05
Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3882e-05 - val_loss: 5.5416e-05
Epoch 299/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3510e-05 - val_loss: 5.5082e-05
Epoch 300/1000
32/32 [==============================] - 0s 125us/step - loss: 5.3141e-05 - val_loss: 5.4746e-05
Epoch 301/1000
32/32 [==============================] - 0s 155us/step - loss: 5.2772e-05 - val_loss: 5.4

...

Epoch 737/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7485e-06 - val_loss: 7.8447e-06
Epoch 738/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7333e-06 - val_loss: 7.8249e-06
Epoch 739/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7187e-06 - val_loss: 7.8063e-06
Epoch 740/1000
32/32 [==============================] - 0s 152us/step - loss: 4.7046e-06 - val_loss: 7.7868e-06
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6894e-06 - val_loss: 7.7657e-06
Epoch 742/1000
32/32 [==============================] - 0s 155us/step - loss: 4.6745e-06 - val_loss: 7.7503e-06
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6610e-06 - val_loss: 7.7278e-06
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6457e-06 - val_loss: 7.7070e-06
Epoch 745/1000
32/32 [==============================] - 0s 155us/step - loss: 4.6327e-06 - val_loss: 7.6

Epoch 811/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9046e-06 - val_loss: 6.6705e-06
Epoch 812/1000
32/32 [==============================] - 0s 187us/step - loss: 3.8959e-06 - val_loss: 6.6605e-06
Epoch 813/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8879e-06 - val_loss: 6.6470e-06
Epoch 814/1000
32/32 [==============================] - 0s 125us/step - loss: 3.8792e-06 - val_loss: 6.6353e-06
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8712e-06 - val_loss: 6.6223e-06
Epoch 816/1000
32/32 [==============================] - 0s 155us/step - loss: 3.8631e-06 - val_loss: 6.6112e-06
Epoch 817/1000
32/32 [==============================] - 0s 218us/step - loss: 3.8554e-06 - val_loss: 6.5972e-06
Epoch 818/1000
32/32 [==============================] - 0s 218us/step - loss: 3.8471e-06 - val_loss: 6.5864e-06
Epoch 819/1000
32/32 [==============================] - 0s 187us/step - loss: 3.8396e-06 - val_loss: 6.5

Epoch 885/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4311e-06 - val_loss: 5.9190e-06
Epoch 886/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4261e-06 - val_loss: 5.9091e-06
Epoch 887/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4217e-06 - val_loss: 5.9008e-06
Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4168e-06 - val_loss: 5.8932e-06
Epoch 889/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4122e-06 - val_loss: 5.8857e-06
Epoch 890/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4075e-06 - val_loss: 5.8776e-06
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4036e-06 - val_loss: 5.8698e-06
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 3.3986e-06 - val_loss: 5.8620e-06
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 3.3946e-06 - val_loss: 5.8

Epoch 959/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1662e-06 - val_loss: 5.4229e-06
Epoch 960/1000
32/32 [==============================] - 0s 249us/step - loss: 3.1633e-06 - val_loss: 5.4174e-06
Epoch 961/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1608e-06 - val_loss: 5.4139e-06
Epoch 962/1000
32/32 [==============================] - 0s 172us/step - loss: 3.1585e-06 - val_loss: 5.4063e-06
Epoch 963/1000
32/32 [==============================] - 0s 125us/step - loss: 3.1554e-06 - val_loss: 5.4002e-06
Epoch 964/1000
32/32 [==============================] - 0s 125us/step - loss: 3.1529e-06 - val_loss: 5.3969e-06
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1507e-06 - val_loss: 5.3913e-06
Epoch 966/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1483e-06 - val_loss: 5.3869e-06
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1458e-06 - val_loss: 5.3

[array([[ 9.7789073e-01,  3.1077614e-01,  1.3600473e+00,  3.9376128e-01,
         -1.6442481e+00],
        [-1.3262837e+00,  1.1190797e-03, -1.2166705e+00, -5.9596807e-01,
          1.3270082e+00],
        [ 1.2231700e+00,  2.3895773e-01,  4.0979668e-01, -1.0308958e+00,
         -7.9593438e-01]], dtype=float32),
 array([ 0.9002875 , -0.52949697,  0.74071836,  0.16111636,  0.97127897],
       dtype=float32),
 array([[ 0.90178114,  0.6115592 , -1.3764067 , -0.10063545, -1.3784319 ,
          0.5929576 , -1.2537191 ,  0.6695224 , -0.48759633, -0.4865871 ],
        [ 0.3886523 ,  0.31567824, -1.2302135 , -0.9934611 , -0.22252479,
          0.5959271 , -1.2674702 ,  0.5941687 , -0.92638266, -0.3256209 ],
        [ 0.8304324 ,  1.4462708 , -0.26950693, -1.0744411 , -0.71184254,
          1.110068  , -0.35528132,  1.1418787 , -1.2677557 , -1.0520877 ],
        [ 0.46059898,  0.82961637, -1.1993229 ,  0.15102465, -0.71156496,
          0.4186685 , -0.7751445 ,  0.3553583 , -0.41250584, -0.8915

In [55]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_tanh.h5')